In [352]:
import os
import random
import tensorflow as tf
import numpy as np
from skimage import io, filters, transform

In [353]:
path = 'C:/Users/Bar/Desktop/FinalDataset'
classes_count = 51
image_width_pixel = image_height_pixel = 28
image_pixels = image_width_pixel * image_height_pixel

train_vectors = []
validation_vectors=[]
test_vectors = []

In [354]:
def read_image(image_path):
    image = io.imread(image_path, as_grey=True)
    image = transform.resize(image, (image_width_pixel, image_height_pixel))
    image = np.reshape(image, image_width_pixel * image_height_pixel)
    image = image.astype(float)
    image = image / np.max(image)
    image = image.astype(int)
    return image
    

In [355]:
# loop through all of the folders (from 0 to 51)
for label in range(classes_count - 1):
    
    # get all of the files in the current label folder
    current_label_files = [entry.path for entry in os.scandir('{}/{}'.format(path, label)) if entry.is_file()]
    
    # shuffles the label files so it won't be same data everytime
    random.shuffle(current_label_files)
    train_count = int(np.ceil(len(current_label_files) * 0.6))
    validation_count = int(np.ceil(len(current_label_files) * 0.2))
    train_files = current_label_files[:train_count]
    validation_files = current_label_files[train_count:train_count+validation_count]
    test_files = current_label_files[train_count+validation_count:]
    
    for image_path in train_files:
        image = read_image(train_image_path)
        train_vectors.append((np.eye(classes_count)[label], image))
        
    for image_path in validation_files:
        image = read_image(image_path)
        validation_vectors.append((np.eye(classes_count)[label], image))
        
    for image_path in test_files:
        image = read_image(image_path)
        test_vectors.append((np.eye(classes_count)[label], image))

random.shuffle(validation_vectors)
random.shuffle(test_vectors)

In [356]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [357]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, classes_count])

# first layer
x_image = tf.reshape(x, [-1,28,28,1])
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# fully connected
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# readout
W_fc2 = weight_variable([1024, classes_count])
b_fc2 = bias_variable([classes_count])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# train
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
training_epochs = 10
batch_size = 100
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    random.shuffle(train_vectors)
    for epoch in range(training_epochs):
        total_batches = int(np.ceil(len(train_vectors)/batch_size))
        for i in range(total_batch):
            starting_index = i * batch_size
            ending_index = (i + 1) * batch_size
            labels, logits = map(list, zip(*train_vectors[starting_index:ending_index]))
            if i%1000 == 0:
                train_accuracy = accuracy.eval(feed_dict={ x:logits, y_: labels, keep_prob: 1.0})
                print("epoch %d, step %d, training accuracy %g"%(epoch, i, train_accuracy))
            train_step.run(feed_dict={x: logits, y_: labels, keep_prob: 0.5})

    test_labels, test_logits = map(list, zip(*test_vectors))
    print("test accuracy %g"%accuracy.eval(feed_dict={
        x: test_logits, y_: test_labels, keep_prob: 1.0}))

epoch 0, step 0, training accuracy 0.04
epoch 1, step 0, training accuracy 0.04
epoch 2, step 0, training accuracy 0.04
epoch 3, step 0, training accuracy 0.04
